# Improving the Search Index

_(Inspired by and borrowed heavily from: Collective Intelligence - [Luís F. Simões](mailto:luis.simoes@vu.nl). IR version and assignments by J.E. Hoeksema, 2014-11-12. Converted to Python 3 and minor changes by Tobias Kuhn, 2015-10-30.)_

*******

**THIS IS NOT THE FINAL VERSION OF THE DOCUMENT. PLEASE UPDATE TO THE LATEST VERSION BEFORE DOING THE ASSIGNMENTS AT THE BOTTOM.**

This notebook's purpose is to improve the search index and query functions built in the previous notebooks and assignments.

## Loading the Data, Defining some functions

This section is copied from the previous notebook.

In [1]:
import pickle, bz2, re
from collections import namedtuple, defaultdict, Counter
from IPython.display import display, HTML
from math import log10

Summaries_file = 'data/air__Summaries.pkl.bz2'
Abstracts_file = 'data/air__Abstracts.pkl.bz2'

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )
for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )
    
def display_summary( id, extra_text='' ): # For a non-notebook version: see the Discussion Board
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long titles or author lists, and links to the paper's  DOI (when available).
    """
    s = Summaries[ id ]
    
    title = ( s.title if s.title[-1]!='.' else s.title[:-1] )
    title = title[:150].rstrip() + ('' if len(title)<=150 else '...')
    if s.doi!='':
        title = '<a href=http://dx.doi.org/%s>%s</a>' % (s.doi, title)
    
    authors = ', '.join( s.authors[:5] ) + ('' if len(s.authors)<=5 else ', ...')
    
    lines = [
        title,
        authors,
        str(s.year),
        '<small>id: %d%s</small>' % (id, extra_text)
        ]
    
    display( HTML( '<blockquote>%s</blockquote>' % '<br>'.join(lines) ) )
    
def display_abstract( id, highlights=[]): # For a non-notebook version: see the Discussion Board
    """
    Function for displaying an abstract. Includes optional (naive) highlighting
    """
    a = Abstracts[ id ]
    for h in highlights:
        a = re.sub(r'\b(%s)\b'%h,'<mark>\\1</mark>',a, flags=re.IGNORECASE)
    display( HTML( '<blockquote>%s</blockquote' % a ) )

In [2]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

In [3]:
inverted_index = defaultdict(set)

# Takes a while
for (id, abstract) in Abstracts.items():
    for term in preprocess(tokenize(abstract)):
        inverted_index[term].add(id)

In [4]:
# Short (one-liner) versions of and_query and or_query
def or_query(query):
    return reduce(lambda a, e: a.union(e), [inverted_index[term] for term in preprocess(tokenize(query))])

def and_query(query): 
    return reduce(lambda a, e: a.intersection(e), [inverted_index[term] for term in preprocess(tokenize(query))])

## Stemming

As we could see from the results of the last assignment, our simple index doesn't handle punctuation and the difference between singular and plural versions of the same word very well. A possible solution to those issues would be to apply better tokenization and stemming. Fortunately, Python's *NLTK* package provides implementations of these algorithms we can use:

In [5]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import EnglishStemmer
import nltk
nltk.download('punkt')
stemmer = EnglishStemmer()

s = '''Good muffins cost $3.88\nin New York.  Please buy me two of them.\n\nThanks.'''

print(tokenize(s))
print(word_tokenize(s))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Luc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['Good', 'muffins', 'cost', '$3.88\nin', 'New', 'York.', '', 'Please', 'buy', 'me', 'two', 'of', 'them.\n\nThanks.']
['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York', '.', 'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.']


In [6]:
print(stemmer.stem("processes"))

process


## Ranking

Another way to improve our search results is to *rank* them. A possible way to do this is to calculate a score for each document based on the matching terms from the query. One such scoring method is *tf-idf*, as explained in the lecture slides.

In order to quickly calculate the scores for a term/document combination, we'll need quick access to a couple of things:
* tf(t,d) - How often does a term occur in a document
* df(t) - In how many documents does a term occur
* N - The number of documents in our index

In [7]:
from collections import Counter, defaultdict

tf_matrix = defaultdict(Counter)
for (id, abstract) in Abstracts.items():
    tf_matrix[id] = Counter(preprocess(tokenize(abstract)))

def tf(t,d):
    return float(tf_matrix[d][t])

def df(t):
    return float(len(inverted_index[t]))
    
def num_documents():
    return float(len(Abstracts))

In [8]:
print(tf('air',16820458))
print(df('air'))
print(num_documents())

2.0
101905.0
190555.0


Using these three helper functions, we can now easily calculate the tf.idf weights of a term in a document by implementing the weighting formula from the slides, which you will do in the assignments below.

## Assignments

* Change (in the code cell below) the *smarter_tokenize* function to use NLTK's word_tokenize function for tokenization, and the *smarter_preprocess* function to perform stemming in addition to case normalization. Does `smarter_and_query("air sample")` return the paper *26488732*? Why (not)?

  <small>We are purposely generating this index on a subset of the data, as generating an index with stemming on the entire set would take up to half an hour</small>

In [11]:
from functools import reduce

def smarter_tokenize(text):
    # Change this
    return word_tokenize(text)

def smarter_preprocess(tokens):
    result = []
    stemmer = EnglishStemmer()
    for token in tokens:
        # Change this
        result.append(stemmer.stem(token))
    return result

def smarter_and_query(query): # Regular and_query using smarter_tokenize and smarter_preprocess
    return reduce(lambda a, e: a.intersection(e), [smarter_index[term] for term in smarter_preprocess(smarter_tokenize(query))])

smarter_index = defaultdict(set)
# Create a subset of around 1400 document IDs
subset = set(Abstracts.keys()).intersection(set(range(26400000,26500000)))

for (id, abstract) in ((k, Abstracts[k]) for k in subset):
    for term in smarter_preprocess(smarter_tokenize(abstract)):
        smarter_index[term].add(id)
        
print(smarter_and_query("air sample"))
print(26488732 in smarter_and_query("air sample"))

{26432020, 26408981, 26419736, 26412057, 26462746, 26480153, 26476059, 26423330, 26460197, 26436142, 26445364, 26428471, 26419772, 26452048, 26434640, 26406492, 26460256, 26475104, 26487904, 26457711, 26464892, 26478716, 26413696, 26487426, 26415752, 26456221, 26452126, 26406048, 26449571, 26436268, 26408120, 26454203, 26425536, 26453185, 26403015, 26489032, 26426569, 26459848, 26473681, 26431700, 26473684, 26434262, 26410714, 26420444, 26451679, 26470625, 26480354, 26449634, 26473705, 26453739, 26484976, 26476807, 26432264, 26434314, 26402063, 26467090, 26404114, 26476306, 26403094, 26448161, 26479908, 26466089, 26479918, 26433327, 26433329, 26437425, 26409267, 26436919, 26444604, 26430286, 26422606, 26426705, 26415447, 26478426, 26443102, 26448746, 26412395, 26484590, 26444147, 26487677, 26408835, 26417541, 26407305, 26452884, 26456986, 26488732, 26414507, 26405807, 26411447, 26445751, 26463685, 26469333, 26459611, 26428398, 26447858, 26439669, 26484214, 26435581}
True


Yes it does, because the word 'sample' is preprocessed to 'sampl' to match all word variations of sample, sampling and samples, which are present in the abstract of the paper.

* Create a function `tfidf(t,d)` that returns the tf.idf score of term `t` in document `d` by using the `tf(t,d)`, `df(t)` and `num_documents()` functions we defined above. The tf.idf formula can be found on the lecture slides.
  
  You can use the `log10(n)` function to calculate log<sub>10</sub>(n), and you can use the code cell below to verify your results.

In [12]:
#print tfidf('embodied',23144668) # 3.35343851032
#print tfidf('evolution',23144668) # 0.302176987782
#print tfidf('notinthedocument',23144668) # 0.0
def tfidf(t, d):
    if(df(t) == 0):
        return 0.0;
    return tf(t, d)*log10(num_documents()/df(t))
print(tfidf('embodied',23144668))
print(tfidf('evolution',23144668))
print(tfidf('notinthedocument',23144668))

0.0
0.0
0.0


* Create a function `query(query_string)`, which accepts as input a single query string that could consist of one or more words, and returns or prints a list of (up to) 10 best matching documents, along with their score. 

  You should use tf.idf to calculate document scores based on the query, and the results should be ordered by score in descending order.

  Hint: Start by copying your `or_query` function from mini-assignment 2, then expand that to rank the results, making use of the `tfidf(t,d)` function you created earlier.

  Use the provided `display_summary(id,extra_text)` function to make the output a bit more 'search engine'-like.

In [13]:
def smarter_or_query(query_string):
    return reduce(lambda a, e: a.union(e), [inverted_index[term] for term in smarter_preprocess(smarter_tokenize(query_string))])

def query(query_string):
    terms = smarter_preprocess(smarter_tokenize(query_string))
    papers = sorted([(paper,sum([tfidf(term, paper) for term in terms])) for paper in smarter_or_query(query_string)], key=lambda value: value[1])
    for paper in reversed(papers[-10:]):
        display_summary(paper[0], ' - tfidf score: %f' % paper[1])
query('air sample')

* Come up with a few example queries to run, and include the output here. Do the results make sense? Why (not)?

In [17]:
print("Results of query \"The Who\"")
query("The Who")

print("Results of query \"Air force\"")
query("Air force")

Results of query "The Who"


Results of query "Air force"


These results don't make a lot of sense for the querys, probably because the words themselves can have multiple meanings. Air force as a commando unit or a force applied by air in the field of physics. There probably are no accurate documents present in the database which refer to the band "The Who" or the air force.